# import pacakage setting

In [10]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import chromedriver_autoinstaller

#multiprocess package
import multiprocessing
from multiprocessing import Pool
import numpy as np
tqdm.pandas()

# Page 1 scrap

### 위스키 증류소 수집 코드

In [154]:
chrome_options = webdriver.ChromeOptions()
chromedriver_autoinstaller.install()                            #크롬 드라이버 자동 설정 : 크롤링시 크롬 드라이버 필요. auto install 사용하여 크롬 버전에 맞는 드라이버 적용

driver = webdriver.Chrome()                                     #웹드라이버중 크롬을 선택하여 driver변수 선언 *driver 변수로 모든 동작을함.
wait = WebDriverWait(driver, 3)       

driver.get('https://www.whiskybase.com/whiskies/distilleries')  #증류소 사이트 접속         
html = driver.page_source                                       #해당 홈페이지의 xml태그 수집
soup = BeautifulSoup(html, 'html.parser')                       #셀레니움으로 가져온 xml테그를 beautifulsoup으로 받아 html.parser로 형식으로 파싱하여 soup 변수에 저장
       
driver.close()                                                  #열려있는 홈페이지 닫기

Exception ignored in: <function Service.__del__ at 0x0000021035CBBDC0>
Traceback (most recent call last):
  File "c:\users\bmsyg\pycharmprojects\pythonproject\venv\lib\site-packages\selenium\webdriver\common\service.py", line 176, in __del__
    self.stop()
  File "c:\users\bmsyg\pycharmprojects\pythonproject\venv\lib\site-packages\selenium\webdriver\common\service.py", line 151, in stop
    self.send_remote_shutdown_command()
  File "c:\users\bmsyg\pycharmprojects\pythonproject\venv\lib\site-packages\selenium\webdriver\common\service.py", line 132, in send_remote_shutdown_command
    if not self.is_connectable():
  File "c:\users\bmsyg\pycharmprojects\pythonproject\venv\lib\site-packages\selenium\webdriver\common\service.py", line 115, in is_connectable
    return utils.is_connectable(self.port)
  File "c:\users\bmsyg\pycharmprojects\pythonproject\venv\lib\site-packages\selenium\webdriver\common\utils.py", line 106, in is_connectable
    socket_ = socket.create_connection((host, port)

KeyboardInterrupt: 

In [ ]:
#위사이트에서 수집해야할 정보를 리스트 선언 및 초기화

distillery_name_list = []          #증류소명 리스트
country_list = []                  #나라명 리스트
whiskies_list = []                 #위스키갯수 리스트
votes_list = []                    #투표수 리스트
rating_list = []                   #투표 평균점수 리스트
link_list = []                     #증류소 링크 리스트

In [ ]:
# soup 변수에 저장된 태그정보(클래스명)에 따라 for문을 통해 진행.  2347 수집시간 대락 30-40min

for i in range(len(soup.select(".clickable"))):
    distillery_name_list.append(soup.select(".clickable")[i].text)
    link_list.append(soup.select(".clickable")[i].a['href']) 
    country_list.append(soup.select(".data")[4*(i+1)].text)
    whiskies_list.append(soup.select(".data")[4*(i+1)+1].text)
    votes_list.append(soup.select(".data")[4*(i+1)+2].text)
    rating_list.append(soup.select(".data")[4*(i+1)+3].text)

In [ ]:
#반복문을 통해 수집된 데이터를 데이터 프레임화 시키기 위해 리스트를 각리스트에 맞는 키(컬럼명으로) 매칭후 json형식으로 변환)

data = {
    'distillery_name' : distillery_name_list,
    'country' : country_list,
    'whiskies' : whiskies_list,
    'votes' : votes_list,
    'rating' : rating_list,
    'link' : link_list
}

In [ ]:
#변환된 data변수를 데이터 프레임화

distillery_table = pd.DataFrame(data)

In [ ]:
#데이터 프레임으로 변환된 데이터 csv로 저장

distillery_table.to_csv('distillery_table_2023_07_25_VOL2.csv',index=False)

# Page 2 Scrap

In [4]:
#저장된 csv파일을 불러오기

distillery_table = pd.read_csv('distillery_table_2023_07_25_VOL2.csv')

In [4]:
#위스키가 존재하지않는 증류소 제거 

distillery_table = distillery_table[distillery_table.whiskies.isna()==False]

### 위스키 정보 링크 수집

In [151]:
# page 2에서는 위스키 정보수집을 위한 링크만 수집하기에 link 리스트만 선인 및 초기화

whikie_link_list = []

In [6]:
#위스키 링크 수집 함수 parameter(distillery_table['link'] 증류소 링크), return : x
#apply를 사용하여 증류소 링크에 들어가있는 위스키 링크를 수집하는 함수입니다.

def scrap_whikies_info(table):
    
    '''
    chrome_options = webdriver.ChromeOptions()              # Page 1 같이 실행시 실행 필요x
    chromedriver_autoinstaller.install()
    
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 3)
    '''
    
    driver.get(table.link)                                  #증류소 링크 하나씩 드라이버에 넘겨 해당 증류소 사이트 접속
    html = driver.page_source                               #해당 페이지에 나와있는 xml태그 html 변수에 저장
    soup = BeautifulSoup(html, 'lxml')                      #soup변수에 lxml를 사용하여 진행 -> html.parser가 아닌 lxml를 사용시 대략 10배 속도 향상
                                                            #https://docs.kanaries.net/ko/tutorials/Python/python-make-beautiful-soup-faster.ko

    
    for i in range(len(soup.select(".photo.buttons"))):     #위스키 사진기준으로 위스키 갯수를 파악후 반복문으로 링크 수집
       '''
        try:                                                # 링크만 수집하여 해당 부분 주석처리
            whikie_img_list.append(soup.select(".photo.buttons")[i].a['href'])      
        except:
            whikie_img_list.append(None)
       '''
        
        link = soup.select(".clickable")                     # clickable이라는 클래명을 가진 태그에서 위스키 링크 추출하여  whikie_link 리스트에 저장
        whikie_link_list.append(link[i]['href'])             #홈페이지 종료
    driver.close()
    

In [12]:
distillery_table.progress_apply(scrap_whikies_info,axis=1)     #apply // progress_apply는 tqdm을 사용하여 얼만큼 진행중인지 모니터링 가능

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 900/900 [7:05:07<00:00, 28.34s/it]


1171    None
1172    None
1173    None
1174    None
1175    None
        ... 
2128    None
2129    None
2130    None
2131    None
2132    None
Length: 900, dtype: object

In [9]:
#scrap_whikies_info로 수집된 정보를 데이터 프레임화 시키기 위해 리스트를 각리스트에 맞는 키(컬럼명으로) 매칭후 json형식으로 변환)

data ={
    'whikie_link' : whikie_link_list
}

NameError: name 'whikie_link_list' is not defined

In [15]:
#변환된 data변수를 데이터 프레임화

whikie_table = pd.DataFrame(data)

In [ ]:
#데이터 프레임으로 변환된 데이터 csv로 저장

whikie_table.to_csv('whikie_table_2023_07_25_VOL2.csv',index=False)

# Page 3 Scrap (작업중...)

In [2]:

whikie_table = pd.read_csv('whikie_table_1100_2000_2023_07_25_vol1.csv')

In [3]:
#위스키 정보 수집

In [4]:
whiskey_name_list = []
whiskey_base_id_list = []
category_list = []
distillery_list = []
bottler_list = []
bottled_list = []
strength_list = []
size_list = []
label_list = []
market_list = []
added_on_list = []
calculated_age_list = []
casknumber_list = []
barcode_list = []
casktype_list = []
bottleing_serie_list = []
number_of_bottles_list = []
vintage_list = []
bottled_for_list = []
over_rating_list = []
votes_list = []
Stated_Age_list = []
Bottle_code_list = [] 

In [5]:
searching_col_dist = {'Whiskybase ID' : whiskey_base_id_list,
'Category' : category_list,
'Distillery' : distillery_list,
'Bottler' : bottler_list,
'Bottled' : bottled_list,
'Strength' : strength_list,
'Size' : size_list,
'Label' : label_list,
'Market' : market_list,
'Added on' : added_on_list,
'Calculated age' : calculated_age_list,
'Casknumber' : casknumber_list,
'Barcode' : barcode_list,
'Casktype' :casktype_list,
'Bottling serie' : bottleing_serie_list,
'Number of bottles' : number_of_bottles_list,
'Vintage' : vintage_list,
'Bottled for' : bottled_for_list,
'Stated Age' : Stated_Age_list,
'Bottle code' : Bottle_code_list}
#'Overall rating' : over_rating_list,
#'votes' : votes_list         }

In [6]:
search_key_list = list(searching_col_dist.keys()) #리스트에 빈값(None) 삽입하기에서 사용하기 위해 변수 선언/

In [7]:
def scrap_whikies_detail_info(table):
    chrome_options = webdriver.ChromeOptions()
    chromedriver_autoinstaller.install()

    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 3)
    
    driver.get(table.whikie_link)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    whiskey_name_list.append(soup.select('.name header h1')[0].text.strip())

    #위스키별 정보 사이트에선 태그구분(클래스명, 아이디명)등으로 구분없이 dt로 되어있어 딕셔너리를 사용하여 키 매칭하여 해당 값을 리스트에 저장
    dt_list = soup.select('dt') #딕셔너리에 대한 key 매칭을 위해 수집
    dd_list = soup.select('dd') #캐에대한 value값
    
    for i in range(len(dt_list)):
       # print(dt_list[i].text.strip())
        if dt_list[i].text.strip() =='Overall rating' or dt_list[i].text.strip() == 'votes'  or dt_list[i].text.strip() == 'vote':  #변경필요 == -> !=으로
            pass
        else:
            searching_col_dist[dt_list[i].text.strip()].append(dd_list[i].text.strip())
    over_rating_list.append(soup.select('dd.votes-rating')[0].text.strip()) #딕셔너리에 넣지않은이유는 vote투표수가 여러개일때와 한개일때의 dt태그명이 다른점과 같은 데이터가 2번 추출된다든점에서 따로 진행
    votes_list.append(soup.select('dd.votes-count')[0].text.strip())


    # 위에 딕셔너리 매칭후 매칭안된 리스트에 빈값(None) 삽입하기
    
    for search_key in search_key_list:
        if (table.name +1) != len(searching_col_dist[search_key]):
            searching_col_dist[search_key].append(None)
    driver.close()
    

#### Multi-process 적용

In [8]:
def parallelize_dataframe(table, func, n_cores=multiprocessing.cpu_count()): #현재 pc에 코어의 갯수를 파악후  n_cores에 저장
    df_split = np.array_split(table, n_cores) #core에 갯수만큼 df를 쪼갠다
    pool = Pool(n_cores)                      #pool을 cpu코어 갯수만큼 생성
    table = pd.concat(pool.map(func,df_split))#나눠진 df를 func(scrap_whikies_detail_info)에 넘겨 진행
    pool.close() 
    pool.join() #모두가 완료될때까지 대기

In [ ]:
parallelize_dataframe(whikie_table,scrap_whikies_detail_info)

In [317]:
#whikie_table.progress_apply(scrap_whikies_detail_info,axis=1)

  0%|                                                                                                                           | 56/63959 [08:34<163:01:16,  9.18s/it]


KeyboardInterrupt: 

In [312]:
data = {'Whiskybase ID' : whiskey_base_id_list,
'Category' : category_list,
'Distillery' : distillery_list,
'Bottler' : bottler_list,
'Bottled' : bottled_list,
'Strength' : strength_list,
'Size' : size_list,
'Label' : label_list,
'Market' : market_list,
'Added on' : added_on_list,
'Calculated age' : calculated_age_list,
'Casknumber' : casknumber_list,
'Barcode' : barcode_list,
'Casktype' :casktype_list,
'Bottling serie' : bottleing_serie_list,
'Number of bottles' : number_of_bottles_list,
'Vintage' : vintage_list,
'Bottled for' : bottled_for_list,
'Stated Age' : Stated_Age_list,
'Bottle code' : Bottle_code_list,
'Overall rating' : over_rating_list,
'votes' : votes_list}

In [313]:
len(Bottle_code_list)

10

In [315]:
whiskey_detail_info = pd.DataFrame(data)